# MIMIC III (Infinite Horizon)

In this notebook, we conducted analysis on the MIMIC III data with infinite horizon. We first analyzed the mediation effect and then evaluate the policy of interest and calculated the optimal policy. As informed by the causal structure learning, here we consider Glucose and PaO2_FiO2 as confounders/states, IV_Input as the action, SOFA as the mediator. 

In [1]:
import os
os.chdir('/nas/longleaf/home/lge/CausalDM')
import pandas as pd
import numpy as np
import pandas as pd
import pickle
file = open('./causaldm/MIMIC3/mimic3_MRL_data_dict_V2.pickle', 'rb')
mimic3_MRL = pickle.load(file)
MRL_df = pd.read_csv('./causaldm/MIMIC3/mimic3_MRL_df_V2.csv')
MRL_df[MRL_df.icustayid==1006]

,icustayid,bloc,Glucose,PaO2_FiO2,IV_Input,SOFA,next_Glucose,next_PaO2_FiO2,Died_within_48H
682,1006,1,91.0,206.000000,0,8,91.0,206.000000,0
683,1006,3,91.0,206.000000,0,8,91.0,206.000000,0
684,1006,6,175.0,100.173913,1,3,175.0,100.173913,0
685,1006,7,175.0,96.000000,1,10,175.0,96.000000,0
686,1006,8,175.0,96.000000,1,9,175.0,96.000000,0
687,1006,10,144.0,187.234036,1,12,144.0,187.234036,-1


## CEL: Mediation Analysis with Infinite Horizon

We processed the MIMIC III data similarly to literature on reinforcement learning by setting the reward of each stage prior to the final stage to 0, and the reward of the final stage to the observed value of Died within 48H. In this section, we analyze the average treatment effect (ATE) of a target policy that provides IV input all of the time compared to a control policy that provides no IV input at all. Using the multiply-robust estimator proposed in [1], we decomposed the ATE into four components, including immediate nature dierct effect (INDE), Immediate nature mediator effect (INME), delayed nature direct effect (DNDE), and delayed nature mediator effect (NDDNME), and estimated each of the effect component. The estimation results are summarized in the table below.

| NIDE           | NIME | NDDE           | NDME           | ATE           |
|---------------|-----|---------------|---------------|---------------|
| -.1453(.0341) | .0046(.0018)   | -.0040(.0060) | .0005(.0018) | -.1441(.0339) |

Specifically, the ATE of the target policy is significantly negative, with an effect size of .1390. Diving deep, we find that the DNME and DNDE are insignificant, whereas the INDE and INME are all statistically significant. Further, taking the effect size into account, we can conclude that the majority of the average treatment effect is directly due to the actions derived from the target treatment policy, while the part of the effect that can be attributed to the mediators is negligible.

In [2]:
from causaldm.learners.Causal_Effect_Learning.Mediation_Analysis import ME_MDP

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [3]:
# Control Policy
def control_policy(state = None, dim_state=None, action=None, get_a = False):
    # fixed policy with fixed action 0
    if get_a:
        action_value = np.array([0])
    else:
        state = np.copy(state).reshape(-1,dim_state)
        NT = state.shape[0]
        if action is None:
            action_value = np.array([0]*NT)
        else:
            action = np.copy(action).flatten()
            if len(action) == 1 and NT>1:
                action = action * np.ones(NT)
            action_value = 1-action
    return action_value
def target_policy(state, dim_state = 1, action=None):
    state = np.copy(state).reshape((-1, dim_state))
    NT = state.shape[0]
    pa = 1 * np.ones(NT)
    if action is None:
        if NT == 1:
            pa = pa[0]
            prob_arr = np.array([1-pa, pa])
            action_value = np.random.choice([0, 1], 1, p=prob_arr)
        else:
            raise ValueError('No random for matrix input')
    else:
        action = np.copy(action).flatten()
        action_value = pa * action + (1-pa) * (1-action)
    return action_value

In [4]:
#Fixed hyper-parameter--no need to modify
expectation_MCMC_iter = 50
expectation_MCMC_iter_Q3 = expectation_MCMC_iter_Q_diff = 50
truncate = 50
problearner_parameters = {"splitter":["best","random"], "max_depth" : range(1,50)},
dim_state=2; dim_mediator = 1
ratio_ndim = 10
d = 2
L = 5
scaler = 'Identity'

In [16]:
Robust_est = ME_MDP.evaluator(mimic3_MRL, r_model = 'OLS',
                     problearner_parameters = problearner_parameters,
                     ratio_ndim = ratio_ndim, truncate = truncate, l2penalty = 10**(-4),
                     target_policy=target_policy, control_policy = control_policy, 
                     dim_state = dim_state, dim_mediator = dim_mediator, 
                     Q_settings = {'scaler': scaler,'product_tensor': False, 'beta': 3/7, 
                                   'include_intercept': False, 'expectation_MCMC_iter_Q3': expectation_MCMC_iter_Q3, 
                                   'expectation_MCMC_iter_Q_diff':expectation_MCMC_iter_Q_diff, 
                                   'penalty': 10**(-4),'d': d, 'min_L': L, 't_dependent_Q': False},
                     expectation_MCMC_iter = expectation_MCMC_iter,
                     seed = 0, nature_decomp = True, method = 'Robust')

Robust_est.estimate_DE_ME()
Robust_est.est_IDE, Robust_est.IME, Robust_est.DDE, Robust_est.DME, Robust_est.TE

Building 0-th basis spline (total 3 state-mediator dimemsion) which has 3 basis, in total 3 features 
Building 1-th basis spline (total 3 state-mediator dimemsion) which has 3 basis, in total 6 features 
Building 2-th basis spline (total 3 state-mediator dimemsion) which has 3 basis, in total 9 features 


(-0.1452771259348552,
 0.004637499340581467,
 -0.003993095022944491,
 0.0005317328373979431,
 -0.14410098877982028)

In [17]:
Robust_est.IDE_se, Robust_est.IME_se, Robust_est.DDE_se, Robust_est.DME_se, Robust_est.TE_se

(0.03413229284322258,
 0.0018118276672658687,
 0.0059963999590195915,
 0.0017995551161793478,
 0.03387985782346984)

## Reference

[1] Ge, L., Wang, J., Shi, C., Wu, Z., & Song, R. (2023). A Reinforcement Learning Framework for Dynamic Mediation Analysis. arXiv preprint arXiv:2301.13348.